<a href="https://colab.research.google.com/github/daystram/ml-playground/blob/master/03_gym_mountaincar_keras/03_gym_mountaincar_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 Gym MountainCar using Keras

In [1]:
!rm -r ml-playground > /dev/null 2>&1
!git clone https://github.com/daystram/ml-playground.git
!sh ml-playground/helper/setup.sh > /dev/null 2>&1

Cloning into 'ml-playground'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 116 (delta 66), reused 64 (delta 27), pack-reused 0
Receiving objects: 100% (116/116), 44.03 KiB | 805.00 KiB/s, done.
Resolving deltas: 100% (66/66), done.


In [0]:
import sys
sys.path.append('/content/ml-playground')

In [3]:
import gym

env = gym.make("MountainCar-v0")
print(env.observation_space)
print(env.action_space)

Box(2,)
Discrete(3)


In [4]:
import numpy as np

acc_fitness = []
training_data = []

sample_count = 5000
steps = 5000
goal_tres = -190

for i in range(sample_count):
  fitness = 0
  action_seq = []
  prev_observation = env.reset()
  
  for _ in range(steps):
    action = np.random.randint(0, 3)
    action_seq.append([prev_observation, action])
    prev_observation, reward, done, _ = env.step(action)
    if prev_observation[0] > -0.2:
      reward = 1
    fitness += reward
    if done: break
      
  if fitness > goal_tres:
    acc_fitness.append(fitness)
    for seq in action_seq:
      if seq[1] == 2:
        action = [0, 0, 1]
      elif seq[1] == 1:
        action = [0, 1 ,0]
      else:
        action = [1, 0, 0]
      training_data.append([seq[0], action])
env.close()
print("{} samples".format(len(training_data)))
print(np.array(acc_fitness))

3600 samples
[-182. -188. -166. -184. -182. -164. -182. -176. -180. -174. -172. -186.
 -180. -182. -182. -172. -182. -162.]


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

drop_rate = 0.3
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(2,)))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(512, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(3, activation='linear'))

model.compile(loss='mse',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])

model.fit(X, y, epochs=20, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
3600/3600 [==============================] - 1s 257us/step - loss: 0.2383 - acc: 0.3392
Epoch 2/20
3600/3600 [==============================] - 1s 157us/step - loss: 0.2270 - acc: 0.3300
Epoch 3/20
3600/3600 [==============================] - 1s 159us/step - loss: 0.2253 - acc: 0.3422
Epoch 4/20
3600/3600 [==============================] - 1s 159us/step - loss: 0.2245 - acc: 0.3456
Epoch 5/20
3600/3600 [==============================] - 1s 161us/step - loss: 0.2248 - acc: 0.3339
Epoch 6/20
3600/3600 [==============================] - 1s 169us/step - loss: 0.2241 - acc: 0.3383
Epoch 7/20
3600/3600 [==============================] - 1s 162us/step - loss: 0.2238 - acc: 0.3536
Epoch 8/20
3600/3600 [==============================] - 1s 162us/step - loss: 0.2236 - acc: 0.3467
Epoch 9/20
3600/3600 [==============================] - 1s 171us/step - loss: 0.2235 - acc: 0.3431
Epoch 10/20
3600/3600 [==============================] - 1s 1

In [7]:
from helper import nbdisplay as disp

!rm -r video
env = disp.wrap_env(gym.make("MountainCar-v0"))
observation = env.reset()
fitness = 0
  
for _ in range(steps):
  action = np.argmax(model.predict(np.array([observation])))
  observation, reward, done, _ = env.step(action)
  fitness += reward
  if done: break

env.close()
disp.show_video()
print("Video can be viewed in Google Colab, use the link on top of the page to open")
print("------ Agent: Reward {}".format(fitness))  

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


[[-0.42397008  0.        ]]


Video can be viewed in Google Colab, use the link on top of the page to open
------ Agent: Reward -187.0


In [0]:
import pickle
import time

filename = '03-agent-{}.model'.format(time.strftime("%Y%m%d_%H%M%S"))
file = open(filename, 'wb')
pickle.dump(model, file=file)
file.close()